In [ ]:

import matplotlib.pyplot as plt
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import Matern, WhiteKernel
from sklearn.preprocessing import StandardScaler
import scipy.stats as st
import pandas as pd
from sklearn.model_selection import KFold 


from plotly.subplots import make_subplots
import plotly.graph_objects as go



data=pd.read_csv('taskdata.csv')
FAC=np.asarray([data.FAC1_1, data.FAC2_1,data.FAC3_1,data.FAC4_1]).T
GRAD=np.asarray([data.Gradient1, data.Gradient2,data.Gradient3]).T
KeepIndex=~np.isnan(FAC[:,0])


In [ ]:

Tasklabels,Taskindices=np.unique(data.Task_name,return_inverse=True)
FAC_TaskCentres=np.zeros([10,4])
for i in range(10):
    FAC_TaskCentres[i,:]=FAC[Taskindices==i+1,:].mean(axis=0)

Grad_TaskCentres=np.zeros([10,3])
for i in range(10):

    Grad_TaskCentres[i,:]=GRAD[np.ix_(Taskindices==(i+1),[0,1,2])].mean(axis=0)


In [22]:
#Fit whole dataset



fig_estimated_mean = make_subplots(rows=2, cols=2, subplot_titles=("Component 1", "Component 2", "Component 3", "Component 4"),specs=[[{'type': 'surface'}, {'type': 'surface'}],[{'type': 'surface'}, {'type': 'surface'}]])

fig_standard_deviation = make_subplots(rows=2,cols=2, subplot_titles=("Component 1", "Component 2", "Component 3", "Component 4"),specs=[[{'type': 'surface'}, {'type': 'surface'}],[{'type': 'surface'}, {'type': 'surface'}]])



# fig, axs = plt.subplots(1,4,figsize=(16, 4), dpi=150)
# fig2, axs2 = plt.subplots(1,4,figsize=(16, 4), dpi=150)

count = 0
for i in range(2):
    for j in range(2):

        standardscaler=StandardScaler()
        X=Grad_TaskCentres

        y = standardscaler.fit_transform(FAC_TaskCentres[:,count].reshape(-1,1))    


        kernel = 1.0 * Matern(length_scale=0.5, length_scale_bounds=(0.5, 1), nu=2.5) + WhiteKernel(noise_level_bounds=[0.001,0.1],noise_level=0.05)


        gpr = GaussianProcessRegressor(kernel=kernel, random_state=3,normalize_y=False,alpha=0)
        gpr.fit(X, y)


        lim = 0.6
        res = 50
        lin = np.linspace(-lim, lim, res)

        
        x1, x2, x3 = np.meshgrid(lin, lin, lin)

        xx = np.vstack((x1.flatten(), x2.flatten(), x3.flatten())).T

        y_mean, y_sd = gpr.predict(xx, return_std=True)


        fig_estimated_mean.add_trace(go.Volume(
            x=pd.Series(x1.flatten(),name="Gradient 1"),
            y=pd.Series(x2.flatten(),name="Gradient 2"),
            z=pd.Series(x3.flatten(),name="Gradient 3"),
            value=y_mean,
            hoverinfo='skip',
            opacityscale=[[0, 0.8], [0.35, 0],[0.65, 0], [1, 0.8]],
            surface_count=25,
            showlegend=False,
            #showscale=False,
            
            colorbar={"tickmode":"array",'tickvals': [min(y_mean),max(y_mean)],'ticktext': ["Predicted low loading","Predicted high loading"]}

            ),i+1,j+1)

        fig_estimated_mean.add_trace(go.Scatter3d(
            x=Grad_TaskCentres[:,0], 
            y=Grad_TaskCentres[:,1],
            z=Grad_TaskCentres[:,2],
            marker_color=FAC_TaskCentres[:,count],
            text=Tasklabels,mode="markers+text",
            #marker_colorbar={"tickmode":"array",'tickvals': [0,1],'ticktext': ["Low predicted loading","High predicted loading"]}
            showlegend=False,
            ),i+1,j+1)

        #fig_estimated_mean.update_layout(coloraxis_colorbar={"tickmode":"array",'tickvals': [0,1],'ticktext': ["Low predicted loading","High predicted loading"]})
        
        #fig_estimated_mean.update_coloraxes(colorbar=dict(tickmode="array",tickvals=[-1,1],ticktext=["High predicted loading","Low predicted loading"])) 
    

        fig_standard_deviation.add_trace(go.Volume(
            x=pd.Series(x1.flatten(),name="Gradient 1"),
            y=pd.Series(x2.flatten(),name="Gradient 2"),
            z=pd.Series(x3.flatten(),name="Gradient 3"),
            value=y_sd,
            hoverinfo='skip',
            showlegend=False,
            #showscale=False,
            opacityscale=[[0, 0.8], [0.35, 0],[0.65, 0], [1, 0.8]],
            colorbar={"tickmode":"array",'tickvals': [min(y_sd),max(y_sd)],'ticktext': ["Low uncertainty in predicted loading","High uncertainty in predicted loading"]},
            surface_count=25,
            ),i+1,j+1)
        count += 1
        fig_estimated_mean.update(layout_coloraxis_showscale=False)
        # axs[i].scatter(x1, x2, c=y_mean,cmap='bwr',alpha=0.5)

        # axs[i].scatter(Grad_TaskCentres[:,0], Grad_TaskCentres[:, 1],s=25,marker='x',c=FAC_TaskCentres[:,i],cmap='bwr')

        # for j in range(10):
        #     axs[i].text(Grad_TaskCentres[j,0],Grad_TaskCentres[j,1],  Tasklabels[j+1], size=8, zorder=1,color='k') 

        # axs2[i].scatter(x1, x2, c=y_sd,cmap='seismic')
    #fig_estimated_mean.update_coloraxes(colorbar=dict(showticklabels=False,tickmode="array",tickvals=[1,-1],ticktext=["High predicted loading","Low predicted loading"])) 
 
fig_estimated_mean.write_html("estimated_mean.html")
fig_standard_deviation.write_html("standard_deviation.html")

c:\Users\Smallwood Lab\anaconda3\envs\gpr\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.

c:\Users\Smallwood Lab\anaconda3\envs\gpr\lib\site-packages\sklearn\gaussian_process\kernels.py:430: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k2__noise_level is close to the specified upper bound 0.1. Increasing the bound and calling fit again may find a better value.

c:\Users\Smallwood Lab\anaconda3\envs\gpr\lib\site-packages\sklearn\gaussian_process\kernels.py:420: ConvergenceWarning:

The optimal value found for dimension 0 of parameter k1__k2__length_scale is close to the specified lower bound 0.5. Decreasing the bound and calling fit again may find a better value.

c:\Users\Smallwood Lab\anaconda3\envs\gpr\lib\site-packages\sklearn\gaussian_proc

In [ ]:
# Do in 3 gradient dimensions
Tasklabels,Taskindices=np.unique(data.Task_name,return_inverse=True)
FAC_TaskCentres=np.zeros([10,4])
for i in range(10):
    FAC_TaskCentres[i,:]=FAC[Taskindices==i+1,:].mean(axis=0)

Grad_TaskCentres=np.zeros([10,3])
for i in range(10):
    #Grad_TaskCentres[i,:]=GRAD[Taskindices==i+1,(0,2)].mean(axis=0)
    Grad_TaskCentres[i,:]=GRAD[np.ix_(Taskindices==(i+1),[0,1,2])].mean(axis=0)


In [ ]:


#Assess out of sample prediction with 3d gradient

fig, axs = plt.subplots(1,1,figsize=(8, 8), dpi=150)

k = 10
kf = KFold(n_splits=k, random_state=None)
numFac=3

X=Grad_TaskCentres
standardscaler=StandardScaler()
avg_acc_score=np.zeros([numFac])
pred_Fac=np.zeros([10,numFac])
real_Fac=np.zeros([10,numFac])

for i in range(numFac):
    acc_score = []
    y = standardscaler.fit_transform(FAC_TaskCentres[:,i].reshape(-1,1))    

    for train_index , test_index in kf.split(X):

        X_train , X_test = X[train_index,:],X[test_index,:]
        y_train , y_test = y[train_index] , y[test_index]

        kernel = 1.0 * Matern(length_scale=0.5, length_scale_bounds=(0.5, 1), nu=2.5)+ WhiteKernel(noise_level_bounds=[0.001,0.5],noise_level=0.05)
        gpr = GaussianProcessRegressor(kernel=kernel, random_state=3,normalize_y=False,alpha=0.0)

        gpr.fit(X_train, y_train)
        pred_values = gpr.predict(X_test)
        pred_Fac[test_index,i]=pred_values
        real_Fac[test_index,i]=y_test
        acc=np.abs(pred_values-y_test.T).sum()
        acc_score.append(acc)
        
    avg_acc_score[i] = np.median(acc_score)

ax = fig.add_subplot(1, 1, 1, projection='3d')

ax.scatter(pred_Fac[:,0],pred_Fac[:,1],pred_Fac[:,2],c='k',marker='+')
ax.scatter(real_Fac[:,0],real_Fac[:,1],real_Fac[:,2],c='k',marker='o')

for i in range(10):
    ax.plot([pred_Fac[i,0],real_Fac[i,0]],[pred_Fac[i,1],real_Fac[i,1]],[pred_Fac[i,2],real_Fac[i,2]],c='k',marker='')
    ax.text(real_Fac[i,0],real_Fac[i,1],real_Fac[i,2],Tasklabels[i+1],c='k')
    

In [ ]:
np.power(np.power(pred_Fac-real_Fac,2).mean(axis=1),0.5)

In [ ]:
avg_acc_score

In [ ]:

Tasklabels,Taskindices=np.unique(data.Task_name,return_inverse=True)
FAC_TaskCentres=np.zeros([10,4])
for i in range(10):
    FAC_TaskCentres[i,:]=FAC[Taskindices==i+1,:].mean(axis=0)

Grad_TaskCentres=np.zeros([10,3])
for i in range(10):
    #Grad_TaskCentres[i,:]=GRAD[Taskindices==i+1,(0,2)].mean(axis=0)
    Grad_TaskCentres[i,:]=GRAD[Taskindices==i+1,:].mean(axis=0)


In [ ]:
from sklearn.svm import SVC
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import permutation_test_score

k = 4
kernel = 1.0 * Matern(length_scale=0.5, length_scale_bounds=(0.5, 1), nu=2.5) #+WhiteKernel(noise_level_bounds=[0.001,0.5],noise_level=0.1)
kf = KFold(n_splits=k, random_state=None)



gpr = GaussianProcessRegressor(kernel=kernel, random_state=None,normalize_y=True,alpha=0.1)
X=Grad_TaskCentres
#X=FAC_TaskCentres
standardscaler=StandardScaler()
#y = standardscaler.fit_transform(FAC_TaskCentres[:,0].reshape(-1,1))    
y = FAC_TaskCentres[:,0]
#y = standardscaler.fit_transform(Grad_TaskCentres[:,2].reshape(-1,1))    



#score_gradfac, perm_scores_gradfac, pvalue_gradfac = permutation_test_score(gpr, X, y, scoring="neg_root_mean_squared_error", cv=kf, n_permutations=100)
score_gradfac, perm_scores_gradfac, pvalue_gradfac = permutation_test_score(gpr, X, y, scoring="neg_mean_absolute_error", cv=kf, n_permutations=1000)




In [ ]:
score_gradfac


In [ ]:
pvalue_gradfac